## 1. Data augmentation

In [2]:
import os
from keras.preprocessing.image import ImageDataGenerator

folder = './data/files/train_single'
out_folder = './data/files/train_single_aug'

categories_1 = []
categories_2 = []
categories_3 = []
categories_4 = []
categories_5 = []

categories_1_max = 15
categories_2_max = 40
categories_3_max = 150
categories_4_max = 5000

x = 3

for category in os.listdir(folder):
    files = os.listdir(os.path.join(folder, category))
    length = len(files)
    if length <= categories_1_max*x:
        categories_1.append(category)
    elif length <= categories_2_max*x: 
        categories_2.append(category)
    elif length <= categories_3_max*x: 
        categories_3.append(category)
    elif length <= categories_4_max*x: 
        categories_4.append(category)
    else: 
        categories_5.append(category)

print(len(categories_1))
print(len(categories_2))
print(len(categories_3))
print(len(categories_4))
print(len(categories_5))

950
1114
1527
1589
90


In [3]:
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

datagen_aug = ImageDataGenerator(
        rotation_range=10,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True)

In [4]:
def augment_images(batch_size, categories, multiplier):
    for category in categories:
        shutil.copytree(os.path.join(folder, category), os.path.join(out_folder, category))
    i = 0
    class2indices = dict(zip(categories, range(len(categories))))
    indices2class = dict(zip(range(len(categories)), categories))
    length = 0
    for category in categories:
        length += len(os.listdir(os.path.join(folder,category)))
    for img_batch in datagen_aug.flow_from_directory(folder, class_mode = 'sparse', target_size=(180, 180),
            batch_size=batch_size,
            classes = categories):
        for img, ind in zip(img_batch[0], img_batch[1]):
            scipy.misc.imsave(os.path.join(out_folder, indices2class[ind], 'img' + str(i) +'.jpg'), img)
            i+=1
        if i > length*(multiplier-1):
            break

In [ ]:
import scipy
import shutil
batch_size = 256
augment_images(batch_size, categories_1, 8)

Found 25645 images belonging to 950 classes.


/home/pon/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [ ]:
augment_images(batch_size, categories_2, 4)
augment_images(batch_size, categories_3, 2)

In [ ]:
import shutil
for category in categories_4:
    shutil.copytree(os.path.join(folder, category), os.path.join(out_folder, category))

In [8]:
import random
for category in categories_5:
    os.mkdir(os.path.join(out_folder, category))
    files = os.listdir(os.path.join(folder, category))
    for i in range(int(categories_4_max*x)):
        file = files.pop(random.randrange(len(files)))
        shutil.copy(os.path.join(folder, category, file), os.path.join(out_folder, category))

## 2. Under and over sampling

In [3]:
import os
from keras.preprocessing.image import ImageDataGenerator

folder = './data/files/train_small_single'
out_folder = './data/files/train_small_single_UOS'

categories_1 = []
categories_2 = []
categories_3 = []
categories_4 = []
categories_5 = []

x = 1

for category in os.listdir(folder):
    files = os.listdir(os.path.join(folder, category))
    length = len(files)
    if length <= 15*x:
        categories_1.append(category)
    elif length <= 50*x: 
        categories_2.append(category)
    elif length <= 150*x: 
        categories_3.append(category)
    elif length <= 5000*x: 
        categories_4.append(category)
    else: 
        categories_5.append(category)

print(len(categories_1))
print(len(categories_2))
print(len(categories_3))
print(len(categories_4))
print(len(categories_5))

353
304
188
196
13


In [4]:
import shutil

def add_images(categories, multiplier):
    for category in categories:
        shutil.copytree(os.path.join(folder, category), os.path.join(out_folder, category))
        for i in range(multiplier - 1):
            for file in os.listdir(os.path.join(folder, category)):
                shutil.copyfile(os.path.join(folder, category, file), os.path.join(out_folder, category, str(i+1)+'_'+ file))

In [5]:
add_images(categories_1, 4)
add_images(categories_2, 3)
add_images(categories_3, 2)
add_images(categories_4, 1)

In [6]:
import random
for category in categories_5:
    os.mkdir(os.path.join(out_folder, category))
    files = os.listdir(os.path.join(folder, category))
    for i in range(5000*x):
        file = files.pop(random.randrange(len(files)))
        shutil.copy(os.path.join(folder, category, file), os.path.join(out_folder, category))

## X. Other

In [52]:
train_generator_aug = datagen_aug.flow_from_directory(
        folder,
        target_size=(180, 180),
        batch_size=batch_size,
        classes = categories,
        save_to_dir = out_folder,
        save_prefix = 'aug')

valid_generator_aug = datagen_aug.flow_from_directory(
        './data/files/valid',
        target_size=(180, 180),
        batch_size=batch_size,
        classes = small_categories)

batch_size = 16
model.fit_generator(
    train_generator_aug,
    epochs=1,
    steps_per_epoch=1189//batch_size*2,
    validation_data=valid_generator_aug,
    validation_steps=315//batch_size,
    workers = 1)

Epoch 1/1
148/148 [==============================] - 19s - loss: 3.1945 - acc: 0.1216 - val_loss: 3.1704 - val_acc: 0.1382
